In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from numba import jit
import h5py
from matplotlib import pyplot as plt
from findiff import FinDiff
from IPython.display import clear_output

In [3]:
X = 100
T = 100
a1s = np.linspace(1e-4, 1e-1, 10, dtype = 'float64')
a2s = np.linspace(1.0, 10.0, 10, dtype = 'float64')
data = np.zeros((len(a1s) * len(a2s) * X * T, 5), dtype = 'float64')
xs = np.linspace(-1.0, 1.0, X, dtype = 'float64')
ts = np.linspace(0.0, 1.0, T, dtype = 'float64')
dx = xs[1] - xs[0]
dt = ts[1] - ts[0]
d2_dx2 = FinDiff(0, dx, 2)
u_0 = xs * xs * np.cos(np.pi * xs)

In [4]:
xs

array([-1. , -0.5,  0. ,  0.5,  1. ])

In [7]:
x_tiles = np.tile(xs, T)
t_repeats = np.repeat(ts, X)

In [8]:
@jit(forceobj = True)
def u_t(t, u, a1, a2, dx):
    u_xx = d2_dx2(u)
    return (a1 * u_xx) - (a2 * (u ** 3 - u))

In [9]:
with h5py.File('observations_sparse.hf', 'w') as file:
    i = 0
    ob = 1
    for a1 in a1s:
        for a2 in a2s:
            end = i + (X * T)
            data[i:end, 0], data[i:end, 1], data[i:end, 2], data[i: end, 3] = x_tiles, t_repeats, a1, a2
            sol = solve_ivp(u_t, t_span = [0.0, 1.0], t_eval = ts, y0 = u_0, args = (a1, a2, dx)).y.T.flatten()
            data[i:end, 4] = sol
            file['ob' + str(ob)] = data[i:end]
            clear_output(wait = True)
            print(ob)
            ob += 1
            i = end

100
